In [1]:
import pandas as pd
import sentencepiece as spm
import smart_open as sm
import gensim
import numpy as np
import re
from tqdm import tqdm_notebook

In [2]:
def read_data(path='data/unigrams.cyr.lc'):
    dic = []
    with sm.open(path, encoding='utf-8') as f:
        for line in f:
            word, num, num2 = line.strip().split('\t')
            dic.append(word)
    return dic

In [3]:
dic = read_data()
with open('data/dict.txt', 'w', encoding='utf-8') as f:
    for i in tqdm_notebook(dic):
        f.write(i)
        f.write('\n')

In [4]:
dic = set(dic)

In [5]:
data = pd.read_csv("data/negative.csv", sep=';', header=None)

In [6]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [7]:
messages = data[3]

In [8]:
def process_text(text):
    tok = text.split(' ')
    pattern = re.compile("[А-я-0-9]+")
    words = pattern.findall(" ".join(tok))
    tokens = [word.lower() for word in words]
    return " ".join(tokens)

In [9]:
text = []
for i in messages:
    text.append(process_text(i))

In [10]:
alph = 'а, б, в, г, д, е, ё, ж, з, и, й, к, л, м, н, о, п, р, с, т, у, ф, х, ц, ч, ш, щ, ъ, ы, ь, э, ю, я'.split(', ')

In [11]:
def change_word(word):
    if len(word)>3:
        for i in range(len(word)+1):
            for j in alph:
                t = word[:i] + j + word[i:]
                if t in dic:
                    return t
            for j in alph:
                p = word[:i] + j + word[i+1:]
                if p in dic:
                    return p
            g = word[:i] + word[i+1:] in dic
            if g in dic:
                return g
    return ''

In [12]:
op = []
v = []
for i in tqdm_notebook(text):
    for j in i.split():
        if j not in dic:
            res = change_word(j)
            if res != '':
                v.append(res)
                op.append(j)

In [29]:
import collections
count = collections.Counter(v)
print(count.most_common())

[('даааа', 610), ('купец', 558), ('соскучились', 306), ('мхах', 277), ('колею', 240), ('фотки', 215), ('липец', 190), ('пред', 161), ('немеет', 155), ('рутвит', 142), ('проспали', 137), ('универа', 116), ('маня', 112), ('айфона', 108), ('успев', 107), ('ноет', 105), ('карой', 104), ('пизду', 99), ('тупо', 94), ('блэя', 94), ('ебаные', 88), ('весят', 88), ('нотик', 84), ('учись', 80), ('надер', 80), ('умирай', 78), ('охотно', 78), ('пофигу', 78), ('алгебре', 78), ('флягу', 75), ('сашка', 73), ('плинта', 73), ('вера', 72), ('ударила', 71), ('запомни', 71), ('компа', 70), ('печаль', 70), ('смайлика', 69), ('заехали', 67), ('физры', 67), ('майна', 67), ('посыпала', 67), ('волнуясь', 66), ('урну', 65), ('близ', 64), ('общага', 64), ('атаки', 63), ('ваша', 63), ('шлак', 63), ('вообще', 62), ('репетитора', 61), ('линь', 61), ('быть', 60), ('развила', 59), ('стремной', 59), ('верь', 59), ('вопли', 59), ('ээээ', 59), ('раман', 58), ('училку', 57), ('обидной', 57), ('ослом', 55), ('кого', 54), (

In [256]:
spm.SentencePieceTrainer.Train('--input=data/dict.txt \
                               --pad_id=0 \
                               --bos_id=2 \
                               --eos_id=3  \
                               --unk_id=1 \
                               --model_prefix=ws/typo_spm \
                               --vocab_size=50000')

True

In [257]:
proc = spm.SentencePieceProcessor()
proc.Load('ws/typo_spm.model')

True

In [258]:
from gensim.models import Word2Vec

sents = [proc.EncodeAsPieces(line.strip()) 
                     for line in text]
w2v = Word2Vec(sents)

w2v.wv.save_word2vec_format('ws/w2v_vectors.bin')

In [259]:
w2v.wv.most_similar('электромобиль')

KeyError: "word 'электромобиль' not in vocabulary"

In [ ]:
proc.EncodeAsPieces('электромобиль')

In [ ]:
op = []
v = []
for i in tqdm_notebook(text):
    for j in i.split():
        if j not in dic:
            for k in w2v.wv.most_similar(proc.EncodeAsPieces(j)):
                if k[0] in dic:
                    op.append(j)
                    v.append(k[0])
                    break

In [ ]:
def tanimoto(s1, s2):
    a, b, c = len(s1), len(s2), 0.0

    for sym in s1:
        if sym in s2:
            c += 1
            
    return c / (a + b - c)

In [ ]:
tanimoto('привет', 'привяо')

In [243]:
op = []
v = []
for i in tqdm_notebook(text):
    for j in i.split():
        if j not in dic:
            for k in dic:
                maxx = 0
                ind = ''
                if tanimoto(j, k) > 0.5:
                    ind = k
                    break
            op.append(j)
            v.append(ind)

KeyboardInterrupt: 